In [2]:
# Configuration initiale
import sys
from pathlib import Path

# Ajouter le répertoire racine au path pour les imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Current working directory: {Path.cwd()}")


Project root: c:\Projet_AI\Assistant_regulation
Current working directory: c:\Projet_AI\Assistant_regulation\notebook


In [ ]:
# Imports des services
from assistant_regulation.planning.services import (
    RetrievalService,
    ValidationService,
    GenerationService,
    MemoryService,
    ContextBuilderService,
    RerankerService,
)
from assistant_regulation.planning.Orchestrator.modular_orchestrator import ModularOrchestrator

print("✅ Tous les imports réussis !")

In [ ]:
# Test du RetrievalService
print("=== Test RetrievalService ===")

try:
    retrieval_service = RetrievalService()
    print("✅ RetrievalService initialisé")
    
    # Test de recherche
    query = "réglementation R107 bus PMR"
    results = retrieval_service.retrieve(
        query,
        use_images=True,
        use_tables=True,
        top_k=3
    )
    
    print(f"✅ Recherche effectuée pour: '{query}'")
    print(f"Résultats texte: {len(results.get('text', []))}")
    print(f"Résultats images: {len(results.get('images', []))}")
    print(f"Résultats tableaux: {len(results.get('tables', []))}")
    text_chunks = results.get('text')
    images_chunks = results.get('images')
    tables_chunks = results.get('tables')
    print(text_chunks)
    print(images_chunks)
    print(tables_chunks)
    # Afficher un échantillon
    if results.get('text'):
        print("\n📄 Premier résultat texte:")
        first_text = results['text'][0]
        content = first_text.get('content', first_text.get('documents', 'Pas de contenu'))
        print(content[:200] + "..." if len(content) > 200 else content)
        
except Exception as e:
    print(f"❌ Erreur RetrievalService: {e}")


In [ ]:
# Test du RerankerService (nécessite des dépendances optionnelles)
print("=== Test RerankerService ===")
try:
    # Test avec mock si les dépendances ne sont pas installées
    reranker_service = RerankerService(model_name="jina-reranker-m0")
    
    print("✅ RerankerService initialisé avec BGE")
    
    # Test de rerank
    test_chunks = [
        {"content": "La météo est belle aujourd'hui."},
        {"content": "Réglementation R107 pour les véhicules de transport."},
        {"content": "Les bus PMR doivent être accessibles."}
    ]
    
    query = "réglementation bus PMR"
    reranked = reranker_service.rerank_chunks(query, test_chunks, top_k=2)
    
    print(f"✅ Rerank effectué pour: '{query}'")
    print(f"Chunks avant: {len(test_chunks)}")
    print(f"Chunks après: {len(reranked)}")
    
    print("\n🏆 Chunks reclassés:")
    for i, chunk in enumerate(reranked):
        print(f"  {i+1}. {chunk['content']}")
            
        
except Exception as e:
    print(f"❌ Erreur RerankerService: {e}")


In [ ]:
print("=== Test Retrieval + Reranking ===")

query = "Quelle sont les dimensions des mannequins passager règlementaire permettant de vérifier le passage dans le véhicule"

# 1) Récupération des chunks
retrieval_service = RetrievalService()
results = retrieval_service.retrieve(
    query,
    use_images=True,   # on se concentre sur le texte
    use_tables=False,
    top_k=30            # nombre de chunks bruts
)

text_chunks = results.get("text", [])
print(f"Chunks récupérés : {len(text_chunks)}")

# 2) Reranking des chunks
reranker = RerankerService(model_name="jina-reranker-m0")        # clé JINA_API_KEY nécessaire
best_chunks = reranker.rerank_chunks(
    query,
    text_chunks,
    top_k=5                         # nombre de chunks réordonnés à garder
)
# 3) Affichage d'un aperçu
for i, chunk in enumerate(best_chunks, 1):
    content = chunk.get("content") or chunk.get("documents") or "—"
    print(f"\n#{i} — score original : {chunk.get('score', 'N/A')}")
    print(content[:250] + ("…" if len(content) > 250 else ""))

In [ ]:
# Test du RerankerService (nécessite des dépendances optionnelles)
print("=== Test RerankerService ===")
try:
    # Test avec mock si les dépendances ne sont pas installées
    reranker_service = RerankerService(model_name="jina-reranker-m0")
    
    print("✅ RerankerService initialisé avec BGE")
    
    # Test de rerank
    test_chunks = [
        {"content": "La météo est belle aujourd'hui."},
        {"content": "Réglementation R107 pour les véhicules de transport."},
        {"content": "Les bus PMR doivent être accessibles."}
    ]
    
    query = "réglementation bus PMR"
    reranked = reranker_service.rerank_chunks(query, test_chunks, top_k=2)
    
    print(f"✅ Rerank effectué pour: '{query}'")
    print(f"Chunks avant: {len(test_chunks)}")
    print(f"Chunks après: {len(reranked)}")
    
    print("\n🏆 Chunks reclassés:")
    for i, chunk in enumerate(reranked):
        print(f"  {i+1}. {chunk['content']}")
            
        
except Exception as e:
    print(f"❌ Erreur RerankerService: {e}")


In [6]:
# Test du GenerationService
print("=== Test GenerationService ===")

try:
    generation_service = GenerationService(llm_provider="ollama", model_name="llama3.2")
    print("✅ GenerationService initialisé (Ollama)")
    
    # Test génération simple
    query = "Qu'est-ce qu'un bus PMR ?"
    answer = generation_service.generate_answer(
        query=query,
        context="",
        conversation_context=""
    )
    
    print(f"✅ Réponse générée pour: '{query}'")
    print(f"📝 Réponse: {answer[:300]}{'...' if len(answer) > 300 else ''}")
    
except Exception as e:
    print(f"❌ Erreur GenerationService: {e}")


=== Test GenerationService ===
✅ GenerationService initialisé (Ollama)
✅ Réponse générée pour: 'Qu'est-ce qu'un bus PMR ?'
📝 Réponse: Bonjour !

Un bus PMR est un véhicule conçu pour les personnes handicapées, en particulier celles qui souffrent de troubles du mouvement ou de la coordination.

PMR signifie "Personnes Handicapées Mobilité Réduite". Les bus PMR sont équipés de sièges adaptés et de systèmes d'accessibilité spéciaux p...


In [ ]:
# Test du ValidationService
print("=== Test ValidationService ===")

try:
    validation_service = ValidationService(llm_provider="mistral", model_name="mistral-large-latest")
    print("✅ ValidationService initialisé")
    
    # Créer des chunks de test
    test_chunks = {
        "text": [
            {
                "content": "La réglementation R107 concerne les véhicules de catégorie M2 et M3.",
                "metadata": {"document_id": "R107", "page_no": 1}
            },
            {
                "content": "Le temps qu'il fait aujourd'hui est ensoleillé.",
                "metadata": {"document_id": "weather", "page_no": 1}
            }
        ],
        "images": [],
        "tables": []
    }
    
    query = "réglementation R107 bus"
    validated_chunks = validation_service.validate_chunks(query, test_chunks)
    
    print(f"✅ Validation effectuée pour: '{query}'")
    print(f"Chunks texte avant: {len(test_chunks['text'])}")
    print(f"Chunks texte après: {len(validated_chunks['text'])}")
    
    if validated_chunks['text']:
        print("\n📄 Chunks validés:")
        for i, chunk in enumerate(validated_chunks['text']):
            content = chunk.get('content', '')[:100]
            print(f"  {i+1}. {content}...")
    
except Exception as e:
    print(f"❌ Erreur ValidationService: {e}")


=== Test ValidationService ===
✅ ValidationService initialisé
✅ Validation effectuée pour: 'réglementation R107 bus'
Chunks texte avant: 2
Chunks texte après: 1

📄 Chunks validés:
  1. La réglementation R107 concerne les véhicules de catégorie M2 et M3....


In [ ]:
# Test du MemoryService
print("=== Test MemoryService ===")

try:
    # Utiliser le client du generation_service pour la cohérence
    memory_service = MemoryService(
        session_id="test_session",
        llm_client=generation_service.raw_client if 'generation_service' in locals() else None,
        model_name="llama3.2"
    )
    print("✅ MemoryService initialisé")
    
    # Ajouter des tours de conversation
    memory_service.add_turn(
        "Qu'est-ce que la R107 ?",
        "La R107 est une réglementation automobile concernant les véhicules de transport en commun.",
        metadata={"sources_count": 2}
    )
    
    memory_service.add_turn(
        "Quelles sont les exigences PMR ?",
        "Les exigences PMR incluent l'accessibilité pour les personnes à mobilité réduite.",
        metadata={"sources_count": 1}
    )
    
    print("✅ Tours de conversation ajoutés")
    
    # Récupérer le contexte
    context = memory_service.get_context("Autres détails sur la R107 ?")
    print(f"✅ Contexte récupéré ({len(context)} caractères)")
    print(f"\n💭 Contexte:\n{context[:500]}..." if len(context) > 500 else f"\n💭 Contexte:\n{context}")
    
    # Statistiques
    stats = memory_service.stats()
    print(f"\n📊 Stats: {stats}")
    
except Exception as e:
    print(f"❌ Erreur MemoryService: {e}")


In [ ]:
# Test du ContextBuilderService
print("=== Test ContextBuilderService ===")

try:
    context_builder = ContextBuilderService()
    print("✅ ContextBuilderService initialisé")
    
    # Créer des chunks multimodaux de test
    test_chunks = {
        "text": [
            {"content": "Réglementation R107: véhicules de catégorie M2 et M3."},
            {"documents": "Les bus PMR doivent respecter des normes d'accessibilité."}
        ],
        "tables": [
            {"content": "Tableau des dimensions minimales: Largeur 80cm, Hauteur 190cm"}
        ],
        "images": [
            {"description": "Schéma d'un emplacement PMR dans un bus avec rampe d'accès."}
        ]
    }
    
    context = context_builder.build_context(test_chunks)
    print(f"✅ Contexte construit ({len(context)} caractères)")
    print(f"\n📋 Contexte:\n{context}")
    
except Exception as e:
    print(f"❌ Erreur ContextBuilderService: {e}")


In [ ]:

import os, textwrap
query = "Organic skincare products for sensitive skin"
documents = [
    "Bio-Hautpflege für empfindliche Haut mit Aloe Vera und Kamille: Erleben Sie die wohltuende Wirkung unserer Bio-Hautpflege, speziell für empfindliche Haut entwickelt. Mit den beruhigenden Eigenschaften von Aloe Vera und Kamille pflegen und schützen unsere Produkte Ihre Haut auf natürliche Weise. Verabschieden Sie sich von Hautirritationen und genießen Sie einen strahlenden Teint.",
    "Organic skincare for sensitive skin with aloe vera and chamomile: Imagine the soothing embrace of nature with our organic skincare range, crafted specifically for sensitive skin. Infused with the calming properties of aloe vera and chamomile, each product provides gentle nourishment and protection. Say goodbye to irritation and hello to a glowing, healthy complexion.",
    "New makeup trends focus on bold colors and innovative techniques: Step into the world of cutting-edge beauty with this seasons makeup trends. Bold, vibrant colors and groundbreaking techniques are redefining the art of makeup. From neon eyeliners to holographic highlighters, unleash your creativity and make a statement with every look.",
    
]
top_k = 3
print("Testing Jina RerankerService…")
print(f"Using API key from env: {'JINA_API_KEY' in os.environ}")
reranker = RerankerService()
ranked = reranker.rerank(query, documents, top_k=top_k)
print(ranked)
for i, (doc, score) in enumerate(ranked, start=1):
    print(f"\nRank #{i} (score={score:.4f}):\n")
    print(textwrap.shorten(doc, width=120, placeholder="…")) 

In [3]:
# Test de l'orchestrateur modulaire complet
print("=== Test ModularOrchestrator ===")

try:
    # Initialiser l'orchestrateur
    orchestrator = ModularOrchestrator(
        llm_provider="mistral",
        model_name="mistral-medium",
        enable_verification=False  # Désactiver pour accélérer les tests
    )
    print("✅ ModularOrchestrator initialisé")
    
    # Test 1: Question générale (pas de RAG)
    print("\n--- Test 1: Question générale ---")
    query1 = "Bonjour, comment allez-vous ?"
    result1 = orchestrator.process_query(
        query1,
        use_images=False,
        use_tables=False,
        top_k=3
    )
    
    print(f"Query: {query1}")
    print(f"Analysis: {result1['analysis']}")
    print(f"Answer: {result1['answer'][:200]}...")
    print(f"Sources: {len(result1['sources'])}")
    
except Exception as e:
    print(f"❌ Erreur Test 1: {e}")


=== Test ModularOrchestrator ===
✅ ModularOrchestrator initialisé

--- Test 1: Question générale ---
Query: Bonjour, comment allez-vous ?
Analysis: {'needs_rag': False, 'query_type': 'general', 'confidence': 0.97, 'context_hint': 'Salutation', 'contains_url': False, 'urls': []}
Answer: Bonjour ! Je vais très bien, merci. Et vous, comment allez-vous aujourd'hui ? 😊...
Sources: 0


In [4]:
try:
    # Test 2: Question réglementaire (avec RAG)
    print("\n--- Test 2: Question réglementaire ---")
    query2 = "Quelles sont les exigences de la réglementation R107 pour les bus PMR ?"
    result2 = orchestrator.process_query(
        query2,
        use_images=True,
        use_tables=True,
        top_k=15
    )
    
    print(f"Query: {query2}")
    print(f"Analysis: {result2['analysis']}")
    print(f"Answer: {result2['answer'][:1000]}...")
    print(f"Sources: {len(result2['sources'])}")
    print(f"Images: {len(result2['images'])}")
    print(f"Tables: {len(result2['tables'])}")
    
except Exception as e:
    print(f"❌ Erreur Test 2: {e}")



--- Test 2: Question réglementaire ---
[{'type': 'structural', 'document': 'docling_core.transforms.chunker.DocMeta', 'regulation': 'UNKNOWN', 'page': 19, 'content': '6. Specifications\nM1, Vehicles used for the carriage of passengers = PMR ≤ 120. M1, Limit Values (dB(A)).Phase 1 = 72. M1, Limit Values (dB(A)).Phase 2 = 70. M1, Limit Values (dB(A)).Phase 3 = 68. M1, Vehicles used for the carriage of passengers = 120 < PMR ≤ 160. M1, Limit Values (dB(A)).Phase 1 = 73. M1, Limit Values (dB(A)).Phase 2 = 71. M1, Limit Values (dB(A)).Phase 3 = 69. M1, Vehicles used for the carriage of passengers = PMR > 160. M1, Limit Values (dB(A)).Phase 1 = 75. M1, Limit Values (dB(A)).Phase 2 = 73. M1, Limit Values (dB(A)).Phase 3 = 71. M1, Vehicles used for the carriage of passengers = PMR > 200, no. of seats ≤ 4, R-point height < 450mm from the ground. M1, Limit Values (dB(A)).Phase 1 = 75. M1, Limit Values (dB(A)).Phase 2 = 74. M1, Limit Values (dB(A)).Phase 3 = 72. M2, Vehicles used for the carriag

In [ ]:
try:
    # Test 3: Question de suivi (avec contexte mémoire)
    print("\n--- Test 3: Question de suivi ---")
    query3 = "Pouvez-vous me donner plus de détails sur ce sujet ?"
    result3 = orchestrator.process_query(
        query3,
        use_conversation_context=True,
        top_k=2
    )
    
    print(f"Query: {query3}")
    print(f"Analysis: {result3['analysis']}")
    print(f"Answer: {result3['answer'][:200]}...")
    
except Exception as e:
    print(f"❌ Erreur Test 3: {e}")


In [ ]:
# Test de performance
import time

print("=== Test de Performance ===")

if 'orchestrator' in locals():
    queries = [
        "Qu'est-ce qu'un PMR ?",
        "Réglementation R107 bus",
        "Exigences accessibilité"
    ]
    
    total_time = 0
    
    for i, query in enumerate(queries, 1):
        start_time = time.time()
        
        try:
            result = orchestrator.process_query(
                query,
                use_images=False,  # Désactiver pour la vitesse
                use_tables=False,
                top_k=2
            )
            
            elapsed = time.time() - start_time
            total_time += elapsed
            
            print(f"Query {i}: {elapsed:.2f}s - {query}")
            print(f"  → RAG: {'✅' if result['analysis']['needs_rag'] else '❌'}")
            print(f"  → Sources: {len(result['sources'])}")
            
        except Exception as e:
            print(f"❌ Erreur Query {i}: {e}")
    
    print(f"\n⏱️ Temps total: {total_time:.2f}s")
    print(f"⏱️ Temps moyen: {total_time/len(queries):.2f}s")
else:
    print("❌ Orchestrateur non disponible pour les tests de performance")


In [ ]:
# Résumé final
print("=== Résumé des Tests ===")
print("\n✅ Services testés:")
print("  • RetrievalService - Recherche multi-sources")
print("  • GenerationService - Génération LLM")
print("  • ValidationService - Filtrage des chunks")
print("  • MemoryService - Mémoire conversationnelle")
print("  • ContextBuilderService - Construction du contexte")
print("  • RerankerService - Réordonnancement (optionnel)")
print("  • ModularOrchestrator - Coordination")

print("\n🎯 Avantages de l'architecture modulaire:")
print("  • Services isolés et testables")
print("  • Injection de dépendances")
print("  • Orchestrateur <200 lignes")
print("  • Facilité de maintenance")

print("\n📋 Prochaines étapes:")
print("  • Installer dépendances reranker si besoin")
print("  • Ajuster les paramètres selon vos besoins")
print("  • Intégrer dans votre application")
print("  • Ajouter tests unitaires")

print("\n🚀 Architecture modulaire opérationnelle !")
